## File purpose: 
### Cluster all the parking spots in to groups of parking spots, minimize the parking spots amount according to their location. 

#### Loading

In [1]:
# Run it to understand if the scooters karnel is working
'''
import geopandas as gpd
print(gpd.__version__)
'''

'\nimport geopandas as gpd\nprint(gpd.__version__)\n'

In [2]:
import datetime as datetime
import pandas as pd
import json
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime,timedelta
import numpy as np

In [3]:
parking_spots_df = pd.read_csv(r'C:\Users\aviyaoh\Downloads\csv_and_merge_files\Parking spots\English GIS initial parking spots.csv') #GIS file

In [4]:
clean_parking_spots_df=parking_spots_df.iloc[:,:3] #remove useless columns
clean_parking_spots_df

,X,Y,Address
0,34.786198,32.070174,Osvaldo Arania 27
1,34.786199,32.070197,HaArbaa 28
2,34.786200,32.070224,HaArbaa 28


In [5]:
clean_parking_spots_df.insert(0, 'ID', ['Spot ' + str(i) for i in range(1, len(clean_parking_spots_df)+1)]) #add ID column

In [6]:
clean_parking_spots_df.columns

Index(['ID', 'X', 'Y', 'Address'], dtype='object')

#### Calculate Distance between two spots

In [7]:
parking_places = clean_parking_spots_df.values.tolist()

In [8]:
from math import radians, cos, sin, asin, sqrt

# Define the Haversine formula to calculate distance between two points
def haversine(lon1, lat1, lon2, lat2):
    # convert degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # radius of Earth in kilometers
    return c * r

In [9]:
for i, j in [(0,1), (0,2), (1,2)]:
    x1, y1 = clean_parking_spots_df.loc[i, ['X','Y']]
    x2, y2 = clean_parking_spots_df.loc[j, ['X','Y']]
    d = haversine(x1, y1, x2, y2)
    print(f"distance between spot {i+1} and spot {j+1}: {d*1000:.2f} meters")

distance between spot 1 and spot 2: 2.56 meters
distance between spot 1 and spot 3: 5.56 meters
distance between spot 2 and spot 3: 3.00 meters


#### clustering of parking spots

In [10]:
def merge_parking_spots(df, max_distance):
    merged_spots = []
    merged_spots_corr=[]
    already_merged_spots_num=[]
    for i, spot1 in df.iterrows():
        if i in already_merged_spots_num: # skip already merged spots
            continue
        merged = [i]
        merged_corr=[(spot1['X'], spot1['Y'])]
        for j, spot2 in df.iterrows():
            if j <= i or j in already_merged_spots_num: # skip previous or already merged spots
                continue
            dist = haversine(spot1['X'], spot1['Y'], spot2['X'], spot2['Y'])
            if dist <= max_distance:
                merged.append(j)
                merged_corr.append((spot2['X'], spot2['Y']))
        merged_spots_corr.append(merged_corr)
        already_merged_spots_num.extend(merged)
        merged_spots.append(merged)
        # if len(merged) > 1:
        #     print(f"New merged parking spot {merged}: {len(merged)} spots")
    return merged_spots,merged_spots_corr

In [11]:
merged_spot_numbers,merged_spots_corr=(merge_parking_spots(clean_parking_spots_df,0.01)) #10m distance for merging

#### find avarage corr for each clusteing

In [12]:
def average_coordinate(points):
    """
    Calculate the average coordinate of a list of points.

    Arguments:
    points -- a list of points, where each point is represented as a tuple of (x, y) coordinates.

  """
    num_points = len(points)
    sum_x = sum_y = 0
    for x, y in points:
        sum_x += x
        sum_y += y
    avg_x = sum_x / num_points
    avg_y = sum_y / num_points
    return avg_x, avg_y

#### create new parking spots df

In [13]:
parking_spots=[['Spot ' + str(spot_number),*average_coordinate(points_list)]for (spot_number,points_list) in zip(range(1, len(merged_spots_corr)+1),merged_spots_corr)]
parking_spots

[['Spot 1', 34.786199, 32.07019833333334]]

In [14]:
#find the 2 edges original parking spots (2 farthest original parking spots from each other)
#creating data frame instead of printing the results
result_data = []

#Loop through every merged parking spot
for i in range(len(merged_spots_corr)):
    # Initialize the maximum distance and the farthest spots
    max_dist = 0
    farthest_spots = None
    orig_spot_1 = None
    orig_spot_2 = None
    parking_spots_num = len(merged_spots_corr[i])
    
    #Loop through every pair of original parking spots in the merged parking spot
    for j in range(len(merged_spots_corr[i])):
        #j+1 ensures that each unique pair is considered.
        #It avoids redundant comparisons and eliminates the need to check for cases where j equals k to prevent comparing a spot with itself
        for k in range(j+1, len(merged_spots_corr[i])):
            # Calculate the distance between the pair using haversine function
            dist = haversine(merged_spots_corr[i][j][0], merged_spots_corr[i][j][1], merged_spots_corr[i][k][0], merged_spots_corr[i][k][1])
            
            #Update the maximum distance and the farthest spots if the current distance is larger
            if dist > max_dist:
                max_dist = dist
                farthest_spots = (parking_places[merged_spot_numbers[i][j]-1], parking_places[merged_spot_numbers[i][k]-1])
                orig_spot_1 = merged_spots_corr[i][j]
                orig_spot_2 = merged_spots_corr[i][k]

    result_data.append({
        'Merged Spot Number': i + 1,
        'Number of Parking Spots': parking_spots_num,
        'Farthest Spot 1': farthest_spots[0][0] if farthest_spots is not None else None,
        'Farthest Spot 2': farthest_spots[1][0] if farthest_spots is not None else None,
        'Distance Between': max_dist if farthest_spots is not None else None,
        'Distance Farthest Spot 1 to Average Merged Spot': haversine(orig_spot_1[0], orig_spot_1[1], parking_spots[i][1], parking_spots[i][2]) if farthest_spots is not None else None,
        'Distance Farthest Spot 2 to Average Merged Spot': haversine(orig_spot_2[0], orig_spot_2[1], parking_spots[i][1], parking_spots[i][2]) if farthest_spots is not None else None
    })

result_df = pd.DataFrame(result_data)
result_df

,Merged Spot Number,Number of Parking Spots,Farthest Spot 1,Farthest Spot 2,Distance Between,Distance Farthest Spot 1 to Average Merged Spot,Distance Farthest Spot 2 to Average Merged Spot
0,1,3,Spot 3,Spot 2,0.005563,0.002707,0.002856


In [15]:
#find the 2 edges original parking spots (2 farthest original parking spots from each other)
#loop through every merged parking spot
for i in range(len(merged_spots_corr)):
    # initialize the maximum distance and the farthest spots
    max_dist = 0
    farthest_spots = None
    #loop through every pair of original parking spots in the merged parking spot
    for j in range(len(merged_spots_corr[i])):
        #j+1 ensures that each unique pair is considered.
        #it avoids redundant comparisons and eliminates the need to check for cases where j equals k to prevent comparing a spot with itself
        for k in range(j+1, len(merged_spots_corr[i])):
            # calculate the distance between the pair using haversine function
            dist = haversine(merged_spots_corr[i][j][0], merged_spots_corr[i][j][1], merged_spots_corr[i][k][0], merged_spots_corr[i][k][1])
            # update the maximum distance and the farthest spots if the current distance is larger
            if dist > max_dist:
                max_dist = dist
                farthest_spots = (parking_places[merged_spot_numbers[i][j]-1], parking_places[merged_spot_numbers[i][k]-1])
                # save the coordinates of the original parking spots
                orig_spot_1 = merged_spots_corr[i][j]
                orig_spot_2 = merged_spots_corr[i][k]

    #print the maximum distance and the farthest spots
    print(f"The maximum distance between two original parking spots in merged parking spot {i+1} is {max_dist} m.")
    
    if farthest_spots is not None:
        print(f"The farthest spots are {farthest_spots[0][0]} and {farthest_spots[1][0]}.")
        # calculate the distance between the first original parking spot and the average coordinates of the merged parking spot using haversine function
        dist1 = haversine(orig_spot_1[0], orig_spot_1[1], parking_spots[i][1], parking_spots[i][2])
        print(f"The distance between the original parking spot {farthest_spots[0][0]} and the average coordinates of merged parking spot {i+1} is {dist1:.6f} m.")
        # calculate the distance between the second original parking spot and the average coordinates of the merged parking spot using haversine function
        dist2 = haversine(orig_spot_2[0], orig_spot_2[1], parking_spots[i][1], parking_spots[i][2])
        print(f"The distance between the original parking spot {farthest_spots[1][0]} and the average coordinates of merged parking spot {i+1} is {dist2:.6f} m.")
    else:
        print("There is only one original parking spot in this merged parking spot.")

The maximum distance between two original parking spots in merged parking spot 1 is 0.005562939306044701 m.
The farthest spots are Spot 3 and Spot 2.
The distance between the original parking spot Spot 3 and the average coordinates of merged parking spot 1 is 0.002707 m.
The distance between the original parking spot Spot 2 and the average coordinates of merged parking spot 1 is 0.002856 m.


In [16]:
#Create 'clustered address' column
merged_spot_numbers, merged_spots_corr = merge_parking_spots(clean_parking_spots_df, 0.01)  # 10m

cluster_street_names = []

for cluster_indices in merged_spot_numbers:
    unique_streets = (
        clean_parking_spots_df
        .loc[cluster_indices, 'Address']  
        .dropna()
        .astype(str)
        .unique()
    )
    
    # Column Output Example: "Ben Gurion | Arlozorov | Dizengoff"
    combined_street_name = " | ".join(sorted(unique_streets))
    cluster_street_names.append(combined_street_name)

In [17]:
parking_spots = [
    ['Spot ' + str(spot_number), *average_coordinate(points_list), street_name]
    for spot_number, points_list, street_name in zip(
        range(1, len(merged_spots_corr) + 1),
        merged_spots_corr,
        cluster_street_names
    )
]


In [18]:
clean_clusterd_parking_spots_df = pd.DataFrame(
    parking_spots,
    columns=['spot_name', 'X', 'Y', 'cluster_address']
)


In [19]:
clean_clusterd_parking_spots_df.head(5)

,spot_name,X,Y,cluster_address
0,Spot 1,34.786199,32.070198,HaArbaa 28 | Osvaldo Arania 27


#### create data frame

In [20]:
clean_clusterd_parking_spots_df= pd.DataFrame(parking_spots, columns=['spot_name', 'X','Y', 'cluster_address'])

### save to a csv file

In [21]:
clean_clusterd_parking_spots_df.to_csv(
    r'C:\Users\aviyaoh\Downloads\csv_and_merge_files\Parking spots\Clustered_parking_spots.csv',
    index=False
)
